In [1]:
import os
os.chdir("f:\\iNeuron\\Projects\\scania_failures_2")
import pandas as pd
import numpy as np
from pprint import pprint


from src.utils import (load_yaml,save_yaml,save_binary,
                       eval_metrics, parameter_tuning, best_model_finder, 
                       stacking_clf_trainer, voting_clf_trainer, model_trainer, mlflow_logger)
from src.constants import *
from src.utils import load_yaml
from src.components.stage_3_data_split import data_splitting_component
from src.components.stage_4_final_preprocessing import stage_4_final_processing_component
from src.config.configuration_manager import ConfigurationManager
from src.entity.entity_config import (Stage2ProcessingConf,
                                      ModelMetricsConf, 
                                      ModelTrainerConf, 
                                      PreprocessorConf, 
                                      DataSplitConf,
                                      Stage1ProcessingConf)
from src import logger

import optuna

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.utils import shuffle

from src.utils import eval_metrics
# from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer

from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

models = {'Logistic_Regression': LogisticRegression, 
            'SGD_Classifier': SGDClassifier,
            'Random Forest': RandomForestClassifier ,
            'Ada_Boost': AdaBoostClassifier, 
            'Grad_Boost': GradientBoostingClassifier, 
            'Bagging_Classifier': BaggingClassifier, 
            'ExtraTreesClassifier': ExtraTreesClassifier, 
            'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
            'Decision_Tree_Classifier': DecisionTreeClassifier,
            'XGB_Classifier': XGBClassifier,
            'KNN_Classifier': KNeighborsClassifier,
            }

In [2]:
import pandas as pd
data_path = "F:\iNeuron\Projects\scania_failures_2\\artifacts\data\processed\stage_1_processing\preprocessed_train_data.csv"
data = pd.read_csv(data_path)

In [3]:
batch_size = 5000
num_batches = data.shape[0] // batch_size
data_shuffled = shuffle(data, random_state=42)

batch_list = []
# Iterate through batches
for i in range(num_batches):
    # Extract a batch
    start_idx = i * batch_size
    end_idx = ((i + 1) * batch_size)
    # print(f"start_idx: {start_idx}, end_idx: {end_idx}")
    batch_list.append(data_shuffled.iloc[start_idx:end_idx, :])

sum1 = 0
sum0 = 0
for i in range(len(batch_list)):
    print(f"Batch: {i}")
    sum0 += batch_list[i]['class'].value_counts()[0] 
    sum1 += batch_list[i]['class'].value_counts()[1] 
    print(batch_list[i]['class'].value_counts(),'\n')

print(f"Total no of Class 0 in all batches: {sum0}")
print(f"Total no of Class 1 in all batches: {sum1}")
    

Batch: 0
class
0    4913
1      87
Name: count, dtype: int64 

Batch: 1
class
0    4917
1      83
Name: count, dtype: int64 

Batch: 2
class
0    4916
1      84
Name: count, dtype: int64 

Batch: 3
class
0    4916
1      84
Name: count, dtype: int64 

Batch: 4
class
0    4915
1      85
Name: count, dtype: int64 

Batch: 5
class
0    4916
1      84
Name: count, dtype: int64 

Batch: 6
class
0    4910
1      90
Name: count, dtype: int64 

Batch: 7
class
0    4919
1      81
Name: count, dtype: int64 

Batch: 8
class
0    4919
1      81
Name: count, dtype: int64 

Batch: 9
class
0    4919
1      81
Name: count, dtype: int64 

Batch: 10
class
0    4923
1      77
Name: count, dtype: int64 

Batch: 11
class
0    4917
1      83
Name: count, dtype: int64 

Total no of Class 0 in all batches: 59000
Total no of Class 1 in all batches: 1000


In [7]:
# batchwise_accuracies = {}
params = load_yaml(PARAMS_PATH)
accuracies = {}
for keys in models.keys():
    accuracies[keys] = {}
    accuracies[keys]['Optuna'] = []
    accuracies[keys]['HyperOpt'] = []

for key,value in models.items():
    def optuna_objective(trial):
        batch_wise_accuracy = []
        batch_number = 1
        space = {}
        flag = 0
        print(f"\n**************\nTrial_Number: {trial.number}\n**************")
        for key_,value_ in params['optuna'][key].items():
                space[key_] = eval(value_)
        for batch in batch_list:
            x = batch.drop(columns='class')
            y = batch['class']
            print(f"\nBatch {batch_number}")
            # for key,value in models.items():
                       
            pprint(f"\nSpace: {space}",compact=True)
            pipeline = Pipeline(steps = [("KNN_Imputer", KNNImputer()),
                                        ("Robust_Scaler", RobustScaler()),
                                        ("SMOTETomek", SMOTETomek(sampling_strategy = "minority",random_state = 42, n_jobs = -1)),
                                        (f"{key}", value(**space))])
            skf_cv = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 42)
            
            accuracy = cross_val_score(estimator=pipeline,
                                    X= x,
                                    y = y,
                                    scoring = 'accuracy',
                                    cv = skf_cv,
                                    n_jobs = -1,
                                    verbose = 2)
            print(f"\nAccuracy of {key}: {np.mean(accuracy)}")
            batch_wise_accuracy.append(np.mean(accuracy))
            accuracies[key]['Optuna'].append(np.mean(accuracy))
            print(f"\nBatchwise_accuracies: {batch_wise_accuracy}")         
            print(f"\nBatchwise_Median_Accuracy: {np.median(batch_wise_accuracy)}")         
            print(f"\nAccumulated Accuracies:\n{accuracies}\n")
            batch_number += 1

            trial.report(np.mean(accuracy),batch_number)
            if (batch_number > 5) and (batch_number % 2 == 0):
                 trial.study.set_user_attr('median_accuracy',np.median(batch_wise_accuracy))
                 if np.mean(accuracy) < np.median(batch_wise_accuracy):
                      flag = 1
                      break

        if flag == 1:
             raise optuna.TrialPruned()
        return np.mean(batch_wise_accuracy)
    find_params = optuna.create_study(direction = 'maximize',pruner=optuna.pruners.MedianPruner(),sampler = optuna.samplers.TPESampler(constant_liar=True))
    find_params.optimize(func = optuna_objective,
                         n_trials = 3)
    

[2024-01-28 19:48:40,060: INFO: utils: params.yaml yaml_file is loaded]


[I 2024-01-28 19:48:40,064] A new study created in memory with name: no-name-c0e35f13-7efc-48e3-a9d3-29e4a8c91ce8



**************
Trial_Number: 0
**************

Batch 1
"\nSpace: {'penalty': None}"


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   10.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.



Accuracy of Logistic_Regression: 0.46279279558254016

Batchwise_accuracies: [0.46279279558254016]

Batchwise_Median_Accuracy: 0.46279279558254016

Accumulated Accuracies:
{'Logistic_Regression': {'Optuna': [0.46279279558254016], 'HyperOpt': []}, 'SGD_Classifier': {'Optuna': [], 'HyperOpt': []}, 'Random Forest': {'Optuna': [], 'HyperOpt': []}, 'Ada_Boost': {'Optuna': [], 'HyperOpt': []}, 'Grad_Boost': {'Optuna': [], 'HyperOpt': []}, 'Bagging_Classifier': {'Optuna': [], 'HyperOpt': []}, 'ExtraTreesClassifier': {'Optuna': [], 'HyperOpt': []}, 'Hist_Grad_Boost_Classifier': {'Optuna': [], 'HyperOpt': []}, 'Decision_Tree_Classifier': {'Optuna': [], 'HyperOpt': []}, 'XGB_Classifier': {'Optuna': [], 'HyperOpt': []}, 'KNN_Classifier': {'Optuna': [], 'HyperOpt': []}}


Batch 2
"\nSpace: {'penalty': None}"


...


Accuracy of Grad_Boost: 0.9770000381676366

Batchwise_accuracies: [0.9751997979755789, 0.9720004378476045, 0.9765997580795966, 0.9773998381596191, 0.9807997584156638, 0.9764009983117422, 0.9781983579274541, 0.9770000381676366]

Batchwise_Median_Accuracy: 0.9767998981236166

Accumulated Accuracies:
{'Logistic_Regression': {'Optuna': [0.46279279558254016, 0.4274040750073275, 0.37198526681218375, 0.4565850335335094, 0.40799655195011414, 0.43138215094076027, 0.41100531394321377, 0.37338486684415817, 0.4611895147981208, 0.412395672126079, 0.47438819559017364, 0.47601140036098427, 0.462385074005607, 0.4256062352955579, 0.43138215094076027, 0.41100531394321377, 0.37338486684415817, 0.4611895147981208, 0.412395672126079, 0.47438819559017364, 0.47601140036098427, 0.462385074005607, 0.4256062352955579], 'HyperOpt': []}, 'SGD_Classifier': {'Optuna': [0.5031919186390813, 0.45941519979317463, 0.5387783187660187, 0.4933956786073758, 0.4823880361982826, 0.54839596306429, 0.4728127603770962, 0.533403

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   17.5s finished
[I 2024-01-28 21:57:21,039] Trial 2 pruned. 
[I 2024-01-28 21:57:21,041] A new study created in memory with name: no-name-b61dcf8d-26b2-45e1-b4cb-1748093cbae8
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.



Accuracy of Grad_Boost: 0.975800518167675

Batchwise_accuracies: [0.9751997979755789, 0.9720004378476045, 0.9765997580795966, 0.9773998381596191, 0.9807997584156638, 0.9764009983117422, 0.9781983579274541, 0.9770000381676366, 0.975800518167675]

Batchwise_Median_Accuracy: 0.9765997580795966

Accumulated Accuracies:
{'Logistic_Regression': {'Optuna': [0.46279279558254016, 0.4274040750073275, 0.37198526681218375, 0.4565850335335094, 0.40799655195011414, 0.43138215094076027, 0.41100531394321377, 0.37338486684415817, 0.4611895147981208, 0.412395672126079, 0.47438819559017364, 0.47601140036098427, 0.462385074005607, 0.4256062352955579, 0.43138215094076027, 0.41100531394321377, 0.37338486684415817, 0.4611895147981208, 0.412395672126079, 0.47438819559017364, 0.47601140036098427, 0.462385074005607, 0.4256062352955579], 'HyperOpt': []}, 'SGD_Classifier': {'Optuna': [0.5031919186390813, 0.45941519979317463, 0.5387783187660187, 0.4933956786073758, 0.4823880361982826, 0.54839596306429, 0.47281276

[W 2024-01-28 22:00:08,803] Trial 0 failed with parameters: {'n_estimators': 57} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "f:\iNeuron\Projects\scania_failures_2\scania_truck\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_2688\650444866.py", line 31, in optuna_objective
    accuracy = cross_val_score(estimator=pipeline,
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "f:\iNeuron\Projects\scania_failures_2\scania_truck\Lib\site-packages\sklearn\model_selection\_validation.py", line 562, in cross_val_score
    cv_results = cross_validate(
                 ^^^^^^^^^^^^^^^
  File "f:\iNeuron\Projects\scania_failures_2\scania_truck\Lib\site-packages\sklearn\utils\_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "f:\iNeuron

KeyboardInterrupt: 

In [9]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_n_estimators,state
0,0,None,2024-01-28 21:57:21.043754,2024-01-28 22:00:08.803868,0 days 00:02:47.760114,57,FAIL


In [11]:
pprint(accuracies,compact=True)

{'Ada_Boost': {'HyperOpt': [],
               'Optuna': [0.9841993185996655, 0.9847997987437326,
                          0.9863999589037776, 0.986799518847731,
                          0.9877996789597662, 0.9852004389038159,
                          0.9857997188077391, 0.9861999988957791,
                          0.9862003589678222, 0.9858003189278111,
                          0.9887998390718016, 0.985199598735715,
                          0.9835993185516557, 0.9843992786076639,
                          0.9863999589037776, 0.9863994788077198,
                          0.9879995189437504, 0.9860005189838383,
                          0.9855996387757262, 0.983999358591667,
                          0.9843995186556928, 0.9868001189678032,
                          0.9867993988237167, 0.9877995589357519,
                          0.98580055897584, 0.9857997188077391]},
 'Bagging_Classifier': {'HyperOpt': [], 'Optuna': []},
 'Decision_Tree_Classifier': {'HyperOpt': [], 'Optuna': []}

In [ ]:
np.median(accuracies['Logistic_Regression']['Optuna'][:5])

In [ ]:
accuracies['Logistic_Regression']['Optuna'][:6]

In [ ]:
find_params.user_attrs

In [ ]:
find_params.trials_dataframe()

In [ ]:
np.mean(accuracies['Logistic_Regression']['Optuna'][:24])

In [ ]:
np.mean(accuracies['Logistic_Regression']['Optuna'])

In [ ]:
np.mean(list(set(accuracies['Logistic_Regression']['Optuna'])))

In [ ]:
# find_params = optuna.create_study(direction = 'maximize')
# find_params.optimize(func = optuna_objective,
#                      n_trials = 5)
#                     #  n_jobs = -1)

In [ ]:
space = {
    'n_estimators': optuna.distributions.IntDistribution(100, 150),
    'criterion' : optuna.distributions.CategoricalDistribution(['log_loss', 'entropy', 'gini']),
    'max_features' : optuna.distributions.CategoricalDistribution(['sqrt', 'log2', None]),
    'class_weight' : optuna.distributions.CategoricalDistribution(['balanced', 'balanced_subsample'])
    }

pipeline = Pipeline(steps = [("KNN_Imputer", KNNImputer()),
                                    ("Robust_Scaler", RobustScaler()),
                                    ("SMOTETomek", SMOTETomek(sampling_strategy = "minority",random_state = 42, n_jobs = -1)),
                                    (f"Random Forest", RandomForestClassifier(n_jobs=-1).set_params(**space))])
skf_cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

# optuna_search = optuna.integration.OptunaSearchCV(estimator = pipeline,
#                                   param_distributions = {'verbose':optuna.distributions.CategoricalDistribution([True,False])},
#                                   cv = skf_cv,
#                                   n_jobs = -1,
#                                   n_trials = 10,
#                                   random_state = 42,
#                                   refit = True,
#                                   return_train_score = False,
#                                   scoring = 'accuracy',
#                                   verbose = 3)

In [ ]:
pipeline

In [ ]:
x = batch_list[2].drop(columns='class')
y = batch_list[2]['class']
optuna_search.fit(x,y)